# Support Vector Machines com Python

## Importando as bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Obtendo os dados

Usaremos o conjunto de dados integrado do Scikit Learn sobre câncer de mama:

In [ ]:
from sklearn.datasets import load_breast_cancer

In [ ]:
cancer = load_breast_cancer()

O conjunto de dados é fornecido em forma de dicionário:

In [ ]:
cancer.keys()

Podemos obter informações e arrays deste dicionário para configurar nosso dataframe e entender as características:

In [ ]:
print(cancer['DESCR'])

In [ ]:
cancer['feature_names']

## Configurando o DataFrame

In [ ]:
df_feat = pd.DataFrame(cancer['data'],columns=cancer['feature_names'])
df_feat.info()

In [ ]:
cancer['target']

In [ ]:
df_target = pd.DataFrame(cancer['target'],columns=['Cancer'])

Agora vamos verificar o dataframe

In [ ]:
df_feat.join(df_target).head()

# Análise exploratória de dados



Iremos pular a parte de visualização dos dados, pois há muitas características que são difíceis de interpretar se você não tiver conhecimento sobre câncer ou células tumorais.

## Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_feat, df_target, test_size=0.30, random_state=101)
#X_train, X_test, y_train, y_test = train_test_split(df_feat, np.ravel(df_target), test_size=0.30, random_state=101)

# Treinando o Support Vector Classifier

In [ ]:
from sklearn.svm import SVC

In [ ]:
model = SVC()

In [ ]:
model.fit(X_train,y_train)

## Previsões e avaliações

Agora vamos testar o modelo treinado.

In [ ]:
predictions = model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
print(confusion_matrix(y_test,predictions))

In [ ]:
print(classification_report(y_test,predictions))

Modelos podem ser melhorados através de ajuste de parâmetros (normalizar os dados também pode ajudar).

Podemos procurar por parâmetros usando um GridSearch!

# Gridsearch

Encontrar os parâmetros corretos (como quais valores de C ou gama usar) é uma tarefa complicada! Mas, felizmente, podemos ser preguiçosos e tentar várias combinações e ver o que funciona melhor! Essa ideia de criar uma 'grade' de parâmetros tentando todas as combinações possíveis é chamada de Gridsearch. Esse método é comum o suficiente para que o Scikit-learn tenha essa funcionalidade incorporada ao GridSearchCV (o CV significa validação cruzada - Cross Validation).

GridSearchCV pega um dicionário que informa os parâmetros que devem ser testados e um modelo para treinar. A grade de parâmetros é definida como um dicionário, onde as chaves são os parâmetros e os valores são as configurações a serem testadas.

In [ ]:
param_grid = {'C': [0.1,1, 10, 100, 1000], 'gamma': [1,0.1,0.01,0.001,0.0001], 'kernel': ['rbf']} 

In [ ]:
from sklearn.model_selection import GridSearchCV

Uma das grandes vantagens do GridSearchCV é que ele é um meta-estimador. Ele pega um estimador como o SVC e cria um novo estimador, que se comporta exatamente da mesma forma - neste caso, como um classificador. Você deve adicionar **refit=True** e escolher para **verbose** qualquer número que desejar, quanto maior o número, mais detalhado (verbose significa apenas a saída de texto que descreve o processo).

In [ ]:
grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=3)

O que **fit** faz é um pouco mais complicado do que o normal. Primeiro, ele executa o mesmo loop com validação cruzada para encontrar a melhor combinação de parâmetros. Uma vez obtida a melhor combinação, ele executa o ajuste novamente em todos os dados passados ​​para ajuste (sem validação cruzada), para construir um único novo modelo usando a melhor configuração de parâmetro.

In [ ]:
grid.fit(X_train,y_train)

Você pode inspecionar os melhores parâmetros encontrados por GridSearchCV no atributo best_params_ e o melhor estimador no atributo best\_estimator_:

In [ ]:
grid.best_params_

In [ ]:
grid.best_estimator_

Em seguida, você pode executar novamente as previsões nesse objeto de grade, exatamente como faria com um modelo normal.

In [ ]:
grid_predictions = grid.predict(X_test)

In [ ]:
print(confusion_matrix(y_test,grid_predictions))

In [ ]:
print(classification_report(y_test,grid_predictions))